#Engenharia Gramatical

Ano Letivo 22/23

Autor Tiago Baptista

# Aula 2

## Sumário



1.   O que é o lark ?
2.   Principais características
  - construções automáticas
  - ferramentas de visualização
  - suporte de ebnf
4.   lexer em lark
5.   Introdução aos transformers

## Notas
[lark cheatsheet](https://lark-parser.readthedocs.io/en/latest/_static/lark_cheatsheet.pdf)

[Documentação](https://lark-parser.readthedocs.io)

[lark common](https://github.com/lark-parser/lark/blob/master/lark/grammars/common.lark)


### Demo introdução a lark e Tranformers

####Exercício 1
Escreva uma gramática independente de contexto para definir uma lista possivelmente vazia de números inteiros separados por vírgulas.

Deve importar o módulo **Lark** para gerar um parser que ao reconhecer uma frase gere a respetiva árvore de parsing.

Use a função de *pretty print* do módulo *Lark* para imprimir a árvore construida.

Nesta primeira versão não se pretende que faça nenhuma travessia à árvore, 
mas em contrapartida pretende-se que escreva várias versões da GIC para usar extended-BNF (uso de operadores para descrever repetições) e para usar apenas BNF (uso de recursividade par descrever repetições). 




In [1]:
from lark import Lark,Token
from lark.tree import pydot__tree_to_png
from lark import Transformer
from lark import Discard

grammar1 = '''
//Regras Sintaticas
start: PE ( | elementos) PD
elementos : NUMERO (VIR NUMERO)*
// elemento : NUMERO 
//Regras Lexicográficas
NUMERO:"0".."9"+ // [0-9]+
PE:"["
PD:"]"
VIR:","
//Tratamento dos espaços em branco
%import common.WS
%ignore WS
'''
grammar5 = '''
//Regras Sintaticas
start: PE ( |elemento (VIR elemento)*) PD
elemento : NUMERO 
//Regras Lexicograficas
NUMERO:"0".."9"+
PE:"["
PD:"]"
VIR:","
//Tratamento dos espaços em branco
%import common.WS
%ignore WS
'''

grammar2 = '''
//Regras Sintaticas
start: PE elementos PD
elementos: elemento VIR elementos
        |
elemento : NUMERO 
//Regras Lexicograficas
NUMERO:"0".."9"+
PE:"["
PD:"]"
VIR:","
//Tratamento dos espaços em branco
%import common.WS
%ignore WS
'''

grammar3 = '''
//Regras Sintaticas
start: PE elementos PD
elementos: elementos VIR elemento
        |
elemento : NUMERO 
//Regras Lexicograficas
NUMERO:"0".."9"+
PE:"["
PD:"]"
VIR:","
//Tratamento dos espaços em branco
%import common.WS
%ignore WS
'''

grammar4 = '''
//Regras Sintaticas
start: PE ( |elementos) PD
elementos: elemento VIR elementos
         | elemento
elemento : NUMERO 
//Regras Lexicograficas
NUMERO:"0".."9"+
PE:"["
PD:"]"
VIR:","
//Tratamento dos espaços em branco
%import common.WS
%ignore WS
'''

grammar = '''
//Regras Sintaticas
start: PE elemento* PD
elemento : NUMERO (VIR NUMERO)*
//Regras Lexicográficas
NUMERO:"0".."9"+ 
PE:"["
PD:"]"
VIR:","
//Tratamento dos espaços em branco
%import common.WS
%ignore WS
'''
# Soma da lista, encontrar o maior da lista
class ExemploTransformer(Transformer):
  def start(self,elementos):
    print("start")
    
  def elemento(self,elemento):
    print("elemento soma")
    print(sum(elemento))
    
  def NUMERO (self,numero):
    print("numero")
    print(numero)
    return int(numero)
    
  def PE(self,pe):
    print("pe")
    print(pe)
    return str(pe)

  def PD(self,pd):
    print("pd")
    print(pd)
    return str(pd)

  def VIR(self,vir):
    print("vir")
    print(vir)
    return Discard

  pass

frase = "[1,23,345]"

p = Lark(grammar)   #não muito bem
#p = Lark(grammar1)  #recomendada
#p = Lark(grammar5)  #boa
#p = Lark(grammar2)  #incorreta
#p = Lark(grammar3)  #incorreta
#p = Lark(grammar4)   #aceitável

tree = p.parse(frase)
print(tree.pretty())
#for element in tree.children:
  #print(element)


data = ExemploTransformer().transform(tree) # chamar o transformer para obter
#print(data)


Defaulting to user installation because normal site-packages is not writeable
start
  [
  elemento
    1
    ,
    23
    ,
    345
  ]

pe
[
numero
1
vir
,
numero
23
vir
,
numero
345
elemento soma
369
pd
]
start


####Exercício 2
Retome uma das GIC escrita em extended-BNF para definir uma linguagem que permita escrever uma lista possívelmente vazia de números inteiros separados por vírgulas,
mas agora use o respetivo Transformer e escreva funções para visitar os nodos da árvore de parsing e escrever o conteúdo de cada uma.


In [2]:
from lark import Lark
from lark.tree import pydot__tree_to_png
from lark import Transformer
from lark import Discard

class MyTransformer(Transformer):
    def start(self, items):
        output={}
        #output["elementos"] = items[1]
        #soma = sum(items[1])
        #output["soma"] = soma
        print("raiz")
        return (output)

    def elemento(self, elemento):
        print(elemento)
        #r = list(filter(lambda x: x!=',', elemento))
        #return r

    def VIR(self,op):
        print(str(op))
        #return Discard

    def NUMERO(self,numero):
        print(str(numero))

    def PE(self,pe):
        print(str(pe))

    def PD(self,pd):
        print(str(pd))

## Primeiro precisamos da GIC
grammar = '''
start: PE elemento* PD
elemento : primeiro resto 
primeiro : NUMERO
resto : (VIR NUMERO)*
NUMERO:"0".."9"+
PE:"["
PD:"]"
VIR:","

%import common.WS
%ignore WS
'''

frase = "[1,2,3]"
p = Lark(grammar)

parse_tree = p.parse(frase)
print(parse_tree.pretty())

data = MyTransformer().transform(parse_tree)
print(data)


start
  [
  elemento
    primeiro	1
    resto
      ,
      2
      ,
      3
  ]

[
1
,
2
,
3
[Tree(Token('RULE', 'primeiro'), [None]), Tree(Token('RULE', 'resto'), [None, None, None, None])]
]
raiz
{}


#### Exercício 3


Desenvolva uma GIC para definir uma linguagem que permita escrever listas
mistas de números e palavras, de tal forma que as 3 frases abaixo sejam frases
válidas dessa linguagem:

LISTA 1 .

Lista aaa .

Lista 1, 2, agora, 3, 4, fim, 7, 8.

Resolva as seguintes alíneas recorrendo ao uso de Transformer :

(a) Calcule, retorne e imprima quantos elementos existem numa lista.

(b) Identifique, retorne e imprima o elemento que mais se repete numa lista.

(c) Calcule, retorne e imprima a soma de todos os números que se encontrem entre as palavras
agora e fim de uma lista.

(d) Verifique se uma lista é válida de acordo com os seguintes parâmetros :

- Caso a palavra "agora" ocorra então a palavra "fim" também tem de aparecer posteriormente na lista e o contrário também.
- Entre "agora" e "fim" tem de aparecer pelo menos um número.


# Aula 3 e 4

## Exercícios

Exercício 1

###### Nota : Abertura de ficheiros (caminho collab) : /content/sample_data


Desenvolva uma GIC para definir uma linguagem que permita descrever os
alunos (identificados pelo seu nome) de uma turma específica, de tal forma
que a frase abaixo seja uma frase válida dessa linguagem (ignore os \n):
```
TURMA A
ana (12, 13, 15, 12, 13, 15, 14);
joao (9,7,3,6,9);
xico (12,16).
TURMA B
ana (12, 13, 15, 12, 13, 15, 14); 
joao (9,7,3,6,9,12);
xico (12,16).
```
Recorrendo ao uso de lark Transformer :

1.1 Conte e imprima o número de alunos presentes, retornando o valor regra de start.

1.2 Calcule e imprima a média de cada aluno,  retornando o valor regra de start.

1.3 Organize os dados de maneira a que a cada nota corresponda os alunos que
tiraram essa nota, retornando o valor regra de start. Exemplo : ```{'12': {'joao', 'xico', 'ana'}, '13': {'ana'}, '15': {'ana'}, '14': {'ana'}, '9': {'joao'}, '7': {'joao'}, '3': {'joao'}, '6': {'joao'}, '16': {'xico'}}
```

1.4 Crie uma query INSERT em SQL que insira as notas de cada alunos numa
tabela Resultado, com as seguintes colunas : nomeAluno, nota,
dataInsercao, turma. (Nota: Garanta que não existem nomes repetidos dentro da mesma turma). Considere que o campo nota pode ser um array com as notas de cada aluno

1.5 Crie um ficheiro markdown para visualizacao das alineas acima.
Exemplo para a turma A:

~~~
# Visualizador de turmas
## Turma A
### Lista de alunos 
- Ana
- Joao
- Xico
### Notas

| Aluno | Media |
|  --------  |  -------  |
| Ana | 13 
| Joao | 6 |
| Xico | 4 |

~~~

1.6 Crie um ficheiro html para visualizacao das alineas acima. Exemplo :
http://tpcg.io/_7FLIBV 


<h1>Turma A<code><br /></code></h1>
<table style="border-collapse: collapse; width: 100%; height: 81px;" border="1">
<tbody>
<tr style="height: 18px;">
<td style="width: 10%; height: 18px; text-align: center;">Nome</td>
<td style="width: 10%; height: 18px; text-align: center;">Nota1</td>
<td style="width: 10%; height: 18px; text-align: center;">Nota2</td>
<td style="width: 10%; height: 18px; text-align: center;">Nota3</td>
<td style="width: 10%; height: 18px; text-align: center;">Nota4</td>
<td style="width: 10%; height: 18px; text-align: center;">Nota5</td>
<td style="width: 10%; height: 18px; text-align: center;">Nota6</td>
<td style="width: 10%; height: 18px; text-align: center;">Nota7</td>
<td style="width: 10%; height: 18px; text-align: center;">M&eacute;dia</td>
<td style="width: 10%; height: 18px; text-align: center;">&nbsp;</td>
</tr>
<tr style="height: 21px;">
<td style="width: 10%; text-align: center; height: 21px;">ana</td>
<td style="width: 10%; text-align: center; height: 21px;">12</td>
<td style="width: 10%; text-align: center; height: 21px;">13</td>
<td style="width: 10%; text-align: center; height: 21px;">15</td>
<td style="width: 10%; text-align: center; height: 21px;">12</td>
<td style="width: 10%; text-align: center; height: 21px;">13</td>
<td style="width: 10%; text-align: center; height: 21px;">15</td>
<td style="width: 10%; text-align: center; height: 21px;">14</td>
<td style="width: 10%; text-align: center; height: 21px;"><span id="cwos" class="qv3Wpe">13</span></td>
<td style="width: 10%; text-align: center; height: 21px;"><img src="https://html-online.com/editor/tiny4_9_11/plugins/emoticons/img/smiley-embarassed.gif" alt="embarassed" /></td>
</tr>
<tr style="height: 21px;">
<td style="width: 10%; text-align: center; height: 21px;">joao</td>
<td style="width: 10%; text-align: center; height: 21px;">9</td>
<td style="width: 10%; text-align: center; height: 21px;">7</td>
<td style="width: 10%; text-align: center; height: 21px;">3</td>
<td style="width: 10%; text-align: center; height: 21px;">6</td>
<td style="width: 10%; text-align: center; height: 21px;">9</td>
<td style="width: 10%; text-align: center; height: 21px;">-</td>
<td style="width: 10%; text-align: center; height: 21px;">-</td>
<td style="width: 10%; text-align: center; height: 21px;">6</td>
<td style="width: 10%; text-align: center; height: 21px;"><img src="https://html-online.com/editor/tiny4_9_11/plugins/emoticons/img/smiley-cry.gif" alt="cry" /></td>
</tr>
<tr style="height: 21px;">
<td style="width: 10%; text-align: center; height: 21px;">xico</td>
<td style="width: 10%; text-align: center; height: 21px;">12</td>
<td style="width: 10%; text-align: center; height: 21px;">16</td>
<td style="width: 10%; text-align: center; height: 21px;">-</td>
<td style="width: 10%; text-align: center; height: 21px;">-</td>
<td style="width: 10%; text-align: center; height: 21px;">-</td>
<td style="width: 10%; text-align: center; height: 21px;">-</td>
<td style="width: 10%; text-align: center; height: 21px;">-</td>
<td style="width: 10%; text-align: center; height: 21px;">4</td>
<td style="width: 10%; text-align: center; height: 21px;"><img src="https://html-online.com/editor/tiny4_9_11/plugins/emoticons/img/smiley-cry.gif" alt="cry" /></td>
</tr>
</tbody>
</table>

# Aula 5

Funçoes sobre Tree :
- .children
- .data 
- visit
- visit_children

Funções sobre Token :
- .type
- .value

## Interpreter para análise

In [ ]:
from lark import Discard
from lark import Lark,Token,Tree
from lark.tree import pydot__tree_to_png
from lark.visitors import Interpreter

class MyInterpreter(Interpreter):
    def __init__(self):
        self.comprimento = 0
        self.soma = 0


    def start(self, tree):
        print("Entrei na Raiz, vou visitar os Elementos")
        r = self.visit(tree.children[1])
        print("Elementos visitados, vou regressar à main()")
        return (self.comprimento, r)

    def elementos(self, tree):
        print(tree.pretty())
        #print(tree)
        r = self.visit_children(tree)
        #print("Agora vou visitar cada Elemento")
        r=0
        for elemento in tree.children:
          print(elemento)
          if (elemento.data == 'elemento' and type(elemento)==Tree):
            print("Este filho adiciono porque é 1 Elemento")
            r += self.visit(elemento)
        return r
        

    def elemento(self, tree):
        r = self.visit_children(tree)
        print("elemento",r)
        if(r[0].type=='NUMERO'):
          self.comprimento += 1
          return int(r[0])
        else:
          return 0
  

    def vir(self, tree):
      pass

    
## Primeiro precisamos da GIC
grammar = '''
start: PE elementos PD
elementos : elemento (vir elemento)*
vir : VIR
elemento : NUMERO | PALAVRA |ASPAS
NUMERO:"0".."9"+ 
ASPAS: ESCAPED_STRING
PALAVRA:("A".."Z"|"a".."z")+
PE:"["
PD:"]"
VIR:","

%import common.WS
%import common.ESCAPED_STRING
%ignore WS
'''

#frase = "[a,1,2,3,a,4]"
frase = "[10,pala,1,2,3,a,4,outra,\"A\"]"
p = Lark(grammar)
parse_tree = p.parse(frase)

data = MyInterpreter().visit(parse_tree)
print("Número de número ",data[0]," Somatório: ",data[1])

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Entrei na Raiz, vou visitar os Elementos
elementos
  elemento	10
  vir	,
  elemento	pala
  vir	,
  elemento	1
  vir	,
  elemento	2
  vir	,
  elemento	3
  vir	,
  elemento	a
  vir	,
  elemento	4
  vir	,
  elemento	outra
  vir	,
  elemento	"A"

elemento [Token('NUMERO', '10')]
elemento [Token('PALAVRA', 'pala')]
elemento [Token('NUMERO', '1')]
elemento [Token('NUMERO', '2')]
elemento [Token('NUMERO', '3')]
elemento [Token('PALAVRA', 'a')]
elemento [Token('NUMERO', '4')]
elemento [Token('PALAVRA', 'outra')]
elemento [Token('ASPAS', '"A"')]
Tree(Token('RULE', 'elemento'), [Token('NUMERO', '10')])
Este filho adiciono porque é 1 Elemento
elemento [Token('NUMERO', '10')]
Tree(Token('RULE', 'vir'), [Token('VIR', ',')])
Tree(Token('RULE', 'elemento'), [Token('PALAVRA', 'pala')])
Este filho adiciono porque é 1 Elemento
elemento [Token('PALAVRA', 'pala')]
Tree(Token('RULE', 'vir'), [Token('VIR', ','

# Exercícios

Refazer o exercício das turmas utilizando lark Interpreter

In [ ]:
!pip install lark
from lark import Discard
from lark import Lark,Token,Tree
from lark.tree import pydot__tree_to_png
from lark.visitors import Interpreter

class MyInterpreter(Interpreter):
  def turmas(self,listaTurmas):
    #start: turma+
    self.turmasDic = {}
    for turma in listaTurmas.children:
      self.visit(turma)
    print("raiz",self.turmasDic)
    return(self.turmasDic)

  def turma(self,turma): 
    #turma : ("TURMA" MAIUSCULA+ alunos PONTO)
    for elemento in turma.children:
      if (type(elemento) == Tree):
        print("TURMA",turma.children[0])
        alunos = self.visit(elemento)
        self.turmasDic[turma.children[0].value]=alunos
  
  def alunos(self,alunos):
    #alunos: aluno (PONTOVIRGULA aluno)*
    print("Visitar os alunos")
    alunosList = []
    # Visitar todos os alunos
    for element in alunos.children:
      # So visitamos os tipo tree que sao os alunos
      if (type(element) == Tree and element.data == "aluno"):
        aluno = self.visit(element)
        alunosList.append(aluno)
    return alunosList

  def aluno(self,aluno):
    #aluno: NOME "(" notas ")"
    print("Visitar o aluno",aluno.children[0])
    return aluno.children[0].value

grammar = '''
turmas: turma+
turma : ("TURMA" MAIUSCULAS alunos PONTO)
alunos: aluno (PONTOVIRGULA aluno)*
aluno: NOME "(" notas ")"
notas: NUMERO ( "," NUMERO)*
MAIUSCULAS:("A".."Z")+
NOME: ("A".."Z"|"a".."z")+
NUMERO:"0".."9"+
PONTO:"."
PONTOVIRGULA:";"
%import common.WS
%ignore WS
'''

# Definir o texto-fonte (input) a analisar
#frase = "TURMA A ana (12, 13, 15, 12, 13, 15, 14); joao (9,7,3,6,9,12); xico (12,16).TURMA B lucas (12, 13, 15, 12, 13, 15, 14); joana (9,7,3,6,9,12); xica (12,16)."
frase = "TURMA A ana (1,2,3); ze (2,4);rui(12). TURMA PL zeca(2); rita(2,4,6). TURMA EG rosa (10,11,12,13)."

# Inicializar o objeto lark
p = Lark(grammar,start="turmas")
# Criar a parsing tree
parse_tree = p.parse(frase)
# Utilizar o Interpreter para visitar a parsing tree
ptnova=MyInterpreter().visit(parse_tree)

print("Res",ptnova)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached lark-1.1.5-py3-none-any.whl (107 kB)
TURMA A
Visitar os alunos
Visitar o aluno ana
Visitar o aluno ze
Visitar o aluno rui
TURMA PL
Visitar os alunos
Visitar o aluno zeca
Visitar o aluno rita
TURMA EG
Visitar os alunos
Visitar o aluno rosa
raiz {'A': ['ana', 'ze', 'rui'], 'PL': ['zeca', 'rita'], 'EG': ['rosa']}
Res {'A': ['ana', 'ze', 'rui'], 'PL': ['zeca', 'rita'], 'EG': ['rosa']}


# Aula 6

## Exercício

1 - Transforme a seguinte GIC em notação lark :
```
 T  = { '[', ']', STRING, DATA, FICH }
         P = {
         p1: album  : capa "[" pags "]" ccapa
         p2: capa   : titulo autor DATA
         p3: titulo : STRING
         p4: autor  : STRING
         p5: ccapa  : fecho DATA
         p6: fecho  : STRING
         p7: pags   : pag+
         p8: pag    : sep | folha
         p9: sep    : titulo
         p10: folha : foto+
         p11: foto  : FICH legenda
         p12: legenda : STRING
```
Considere a seguinte frase de exemplo válida :
```
        "Um passeio pelo Geres" 
        Joaozinho 
        03-04-2000 
        [ 
          "Cascata do tahiti"
          img1
          "Vista da cascata."
           
          "A aldeia de Soajo"
          img2 
          "casa da aldeia."
        ] 
        "Dedicado à família" 
        03-05-2000
```
2 - Imprimir o album para latex, considerando :

  -  que o titulo deverá ser um \title ;
  -  que o autor deverá ser um \author ;
  -  cada página deverá ser um \title e terminar com o \newpage ;
  - uma foto devera usar o nome do ficheiro para renderizar e a legenda como \caption.Exemplo :
  ```
\begin{figure}[h!]
\centering
\includegraphics[width=0.3\textwidth]{frog.jpg}
\caption{\label{fig:frog}Vista da cascata.}
\end{figure}
```

3 - Extender a linguagem para permitir que uma folha possa também ter texto descritivo do momento retratado na foto.

4 - Atualizar a impressão do livro para latex.